In [1]:
import pandas as pd
import implicit
import os
import numpy as np
from scipy import sparse

Решается задача товарных рекомендаций.

- `purchases_train.csv` - история покупок в розничном магазине (с 21 октября 2003 года по 12 марта 2004 года)

- `purchases_test.csv` - покупки за следующую неделю (с 13 по 19 марта 2004 года). В этой выборке для каждого пользователя исключены товары, которые он уже покупал за период обучающей выборки

- `customers.csv` - пол клиентов

В решении ниже
- обучается модель матричного разложения AlternateLeastSquares
- сравнивается с тестовыми данными
- измеряется ее качество по метрике map@10
- данные о поле клиентов не используются

Улучшите решение с помощью информацию о поле клиентов

In [2]:
purchases_train = pd.read_csv('purchases_train.csv')
purchases_train.head()

,customer_id,product_id,datetime
0,8698595,12530,2004-03-10 22:18:43.497459200
1,13271885,7541,2004-03-06 02:24:43.209763200
2,16852746,13134,2004-03-10 01:03:09.598614400
3,16852746,6572,2004-03-04 16:45:16.522566400
4,14619070,4659,2004-03-12 13:29:35.011481600


Обучаем модель AlternateLeastSquares

In [3]:
user_items = sparse.coo_matrix(
    (
        np.ones(purchases_train.customer_id.size, dtype=np.float32),
        (
            purchases_train.customer_id,
            purchases_train.product_id
        )
    )
).tocsr()

In [4]:
item_users = user_items.T.tocsr()

In [5]:
# в связи с техническими ограничениями:
os.environ["MKL_NUM_THREADS"] = "1" # export MKL_NUM_THREADS=1
factors = 10 # из-за Memory Errors. В original было =64

In [6]:
model = implicit.als.AlternatingLeastSquares(factors=factors, iterations=100)

In [7]:
np.random.seed(42)
model.fit(item_users=item_users)

purchases_test.csv содержит данные о покупках с 13 марта 2004 по 20 марта 2004 - то есть неделя следующая за обучающей выборкой

для каждого пользователя исключены те товары, которые он покупал в обучающей выборке

In [8]:
purchases_test = pd.read_csv('purchases_test.csv')
display(
    purchases_test.head(),
)

,customer_id,product_id,datetime
0,1021292,6197,2004-03-18 13:35:19.145152000
1,11379978,4659,2004-03-19 18:51:31.887936000
2,13271885,5659,2004-03-14 05:47:21.544166400
3,13271885,1015,2004-03-15 14:41:19.702089601
4,12315337,12072,2004-03-19 10:39:17.148105600


Измеряем качество рекомендаций с помощью метрики map@10

In [9]:
relevant = purchases_test.groupby('customer_id')['product_id'].apply(lambda s: s.values).reset_index()
relevant.rename(columns={'product_id': 'product_ids'}, inplace=True)
relevant.head()

,customer_id,product_ids
0,107,[5868]
1,453,[11854]
2,1011,"[10609, 7110]"
3,1135,[8994]
4,2947,[5868]


In [10]:
recommendations = []
for user_id in relevant['customer_id']:
    recommendations.append([x[0] for x in model.recommend(userid=user_id, user_items=user_items, N=10)])

In [11]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if len(actual) == 0:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [12]:
mapk(relevant['product_ids'], recommendations, k=10)

0.14878238193151377

Задание: используйте пол клиентов для улучшения модели

In [14]:
customers = pd.read_csv('customers.csv')
customers.head()

,customer_id,sex
0,14386819,Female
1,1481405,NaN
2,16745074,NaN
3,10325906,NaN
4,11167384,NaN


In [15]:
# добавляем столбец с полом - и выделяем три сегмента: male, female, unknown

purchases_sex_train = pd.merge(purchases_train, customers, on='customer_id', how='left')
purchases_male_train = purchases_sex_train[purchases_sex_train['sex']=='Male']
purchases_female_train = purchases_sex_train[purchases_sex_train['sex']=='Female']
purchases_unknown_train = purchases_sex_train[(purchases_sex_train['sex']!='Female') & (purchases_sex_train['sex']!='Male')]
print("Train Segment Stats:")
print("All: ", purchases_sex_train.size)
print("Female: ", purchases_female_train.size)
print("Male: ", purchases_male_train.size)
print("Unknown: ", purchases_unknown_train.size)
print("Check: ", purchases_sex_train.size - purchases_female_train.size - purchases_male_train.size - purchases_unknown_train.size)


Train Segment Stats:
All:  1406744
Female:  635180
Male:  208080
Unknown:  563484
Check:  0


In [16]:
# Обучаем три модели. По одной на сегмент.

male_items = sparse.coo_matrix(
    (
        np.ones(purchases_male_train.customer_id.size, dtype=np.float32),
        (
            purchases_male_train.customer_id,
            purchases_male_train.product_id
        )
    )
).tocsr()
item_males = male_items.T.tocsr()

female_items = sparse.coo_matrix(
    (
        np.ones(purchases_female_train.customer_id.size, dtype=np.float32),
        (
            purchases_female_train.customer_id,
            purchases_female_train.product_id
        )
    )
).tocsr()
item_females = female_items.T.tocsr()

unknown_items = sparse.coo_matrix(
    (
        np.ones(purchases_unknown_train.customer_id.size, dtype=np.float32),
        (
            purchases_unknown_train.customer_id,
            purchases_unknown_train.product_id
        )
    )
).tocsr()
item_unknowns = unknown_items.T.tocsr()

In [17]:
model_male = implicit.als.AlternatingLeastSquares(factors=factors, iterations=100)
model_female = implicit.als.AlternatingLeastSquares(factors=factors, iterations=100)
model_unknown = implicit.als.AlternatingLeastSquares(factors=factors, iterations=100)

In [18]:
np.random.seed(42)
model_male.fit(item_users=item_males)

100%|██████████| 100.0/100 [00:38<00:00,  2.83it/s]


In [19]:
model_female.fit(item_users=item_females)

100%|██████████| 100.0/100 [00:36<00:00,  2.81it/s]


In [20]:
model_unknown.fit(item_users=item_unknowns)

100%|██████████| 100.0/100 [00:35<00:00,  2.64it/s]


In [21]:
# Добавляем информацию о поле
relevant_seg = pd.merge(relevant, customers, on='customer_id', how='left')
relevant_seg.head()

,customer_id,product_ids,sex
0,107,[5868],NaN
1,453,[11854],Male
2,1011,"[10609, 7110]",Female
3,1135,[8994],Female
4,2947,[5868],Female


In [22]:
# Используем модели по сегметам для предсказаний     
recommendations_seg = []
for user_id in relevant['customer_id']:
    if relevant_seg.loc[relevant_seg.customer_id == user_id].sex.values[0] == 'Female':
        recommendations_seg.append([x[0] for x in model_female.recommend(userid=user_id, user_items=female_items, N=10)])
    else: 
        if relevant_seg.loc[relevant_seg.customer_id == user_id].sex.values[0] == 'Male':
            recommendations_seg.append([x[0] for x in model_male.recommend(userid=user_id, user_items=male_items, N=10)])  
        else:
            recommendations_seg.append([x[0] for x in model_unknown.recommend(userid=user_id, user_items=unknown_items, N=10)])  
    


In [23]:
print('Без сегментов')
print(mapk(relevant['product_ids'], recommendations, k=10))

print('По сегментам')
print(mapk(relevant['product_ids'], recommendations_seg, k=10))

Без сегментов
0.1487813820914882
По сегментам
0.15453599497650544
